# View Campaign and Interactions

In the first notebook `Personalize_BuildCampaign.ipynb` you successfully built and deployed a recommendation model using deep learning with Amazon Personalize.

This notebook will expand on that and will walk you through adding the ability to react to real time behavior of users. If their intent changes while browsing a movie, you will see revised recommendations based on that behavior.

It will also showcase demo code for simulating user behavior selecting movies before the recommendations are returned.

Below we start with just importing libraries that we need to interact with Personalize

In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import uuid

The line below will retrieve your shared variables from the first notebook.

In [2]:
%store -r

In [3]:
# Setup and Config
# Recommendations from Event data
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's Event Streaming
personalize_events = boto3.client(service_name='personalize-events')

## Creating an Event Tracker

Before your recommendation system can respond to real time events you will need an event tracker, the code below will generate one and can be used going forward with this lab. Feel free to name it something more clever.

In [11]:
response = personalize.create_event_tracker(
    name='AllstoreClickTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']

arn:aws:personalize:us-east-1:387269085412:event-tracker/c50b14e7
8b1ce80f-3c86-4924-aaeb-f4a7bb992a53


In [12]:
# Please note one Dataset Group can only have one "event tracker".

In [13]:
event_tracker_arn = response['eventTrackerArn']

## Configuring Source Data

Above you'll see your tracking ID and this has been assigned to a variable so no further action is needed by you. The lines below are going to setup the data used for recommendations so you can render the list of Allstore products later.

In [20]:
# First load items into memory
items = pd.read_csv('s3://personalizedemoaseanunicorngymq419/convertcsv.csv', sep='|', usecols=[0,1], encoding='latin-1', names=['ITEM_ID', 'TITLE'], index_col='ITEM_ID')

def get_product_title(product_id):
    """
    Takes in an ID, returns a title
    """
    product_id = str(product_id)
    return items.loc[product_id]['TITLE']

## Getting Recommendations

First we will render the recommendations again from the previous notebook:

In [21]:
recommendations_df

,Item Description
0,Rothco 550lb. Type III Nylon Paracord
1,Ultralight Backpacking Canister Camp Stove wit...
2,Emergency Mylar Thermal Blankets (Pack of 10)
3,Mirrycle MTB Bar End Mountain Bicycle Mirror
4,GSI Outdoors Glacier Stainless Bottle Cup/Pot
5,Serfas TCPG Bicycle Floor Pump
6,Light My Fire Titanium Spork
7,Outdoor Products 3-Pack Ultimate Dry Sack
8,LifeStraw Personal Water Filter
9,MetroFlash Safety Zone Tail Light


## Simulating User Behavior

The lines below provide a code sample that simulates a user interacting with a particular item, you will then get recommendations that differ from those when you started.

In [22]:
session_dict = {}

In [23]:
def send_product_click(USER_ID, ITEM_ID):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[USER_ID]
    except:
        session_dict[USER_ID] = str(uuid.uuid1())
        session_ID = session_dict[USER_ID]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
    
        
    # Make Call
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= USER_ID,
    sessionId = session_ID,
    eventList = [{
         'sentAt': int(time.time()),
         'eventType': 'EVENT_TYPE',
         'properties': event_json
         }]
)

Immediately below this line will update the tracker as if the user has clicked a particular title.


If the table generated by the cells below does not shift the recommendations simply try another random 3 digit number in the cell above and run both cells again. You'll see a third column generated of recommendations.

In [24]:
# Pick a product_ID, we will use '1881509818'
product_to_click = 1881509818
product_title_clicked = get_product_title(product_to_click)
send_product_click(USER_ID=str(user_id), ITEM_ID=product_to_click)

After executing this block you will see the alterations in the recommendations now that you have event tracking enabled and that you have sent the events to the service.

In [25]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(user_id),
)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

recommendation_list = []

for item in item_list:
    title = get_product_title(item['itemId'])
    recommendation_list.append(title)
    
new_rec_DF = pd.DataFrame(recommendation_list, columns = [product_title_clicked])

recommendations_df = recommendations_df.join(new_rec_DF)
recommendations_df

Recommendations for user:  A10CKY22VNWXHK


,Item Description,Ghost Inc Glock Armorers Tool 3/32 Punch
0,Rothco 550lb. Type III Nylon Paracord,Ultralight Backpacking Canister Camp Stove wit...
1,Ultralight Backpacking Canister Camp Stove wit...,Vitalsox Graduated Compression Performance Pat...
2,Emergency Mylar Thermal Blankets (Pack of 10),Rothco 550lb. Type III Nylon Paracord
3,Mirrycle MTB Bar End Mountain Bicycle Mirror,Fitbit Flex Wireless Activity + Sleep Wristban...
4,GSI Outdoors Glacier Stainless Bottle Cup/Pot,MetroFlash Safety Zone Tail Light
5,Serfas TCPG Bicycle Floor Pump,Mirrycle MTB Bar End Mountain Bicycle Mirror
6,Light My Fire Titanium Spork,"Dr. Bronner 18-in-1 Pure-Castile Soap, Hemp Al..."
7,Outdoor Products 3-Pack Ultimate Dry Sack,Tenn Waterproof Windproof Cycle Cycling Gloves...
8,LifeStraw Personal Water Filter,Shimano Hyperglide Chain (8 &amp; 9 speed)
9,MetroFlash Safety Zone Tail Light,Vanguard Endeavor XF Angled Eyepiece Spotting ...


## Conclusion

You can see now that recommendations are altered by changing the movie that a user interacts with, this system can be modified to any application where users are interacting with a collection of items. These tools are available at any time to pull down and start exploring what is possible with the data you have.

Execute the cell below to store values needed for the cleanup notebook.

Finally when you are ready to remove the items from your account, open the `Cleanup.ipynb` notebook and execute the steps there.


In [26]:
%store event_tracker_arn

Stored 'event_tracker_arn' (str)
